In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv("../../../Divar.csv")

C:\Users\Rumi\AppData\Local\Temp\ipykernel_19960\2225313960.py:6: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../../Divar.csv")


In [2]:
pd.set_option('display.max_columns', None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 61 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Unnamed: 0                  1000000 non-null  int64  
 1   cat2_slug                   1000000 non-null  object 
 2   cat3_slug                   999999 non-null   object 
 3   city_slug                   999998 non-null   object 
 4   neighborhood_slug           437139 non-null   object 
 5   created_at_month            1000000 non-null  object 
 6   user_type                   288882 non-null   object 
 7   description                 1000000 non-null  object 
 8   title                       999946 non-null   object 
 9   rent_mode                   352994 non-null   object 
 10  rent_value                  351322 non-null   float64
 11  rent_to_single              19 non-null       object 
 12  rent_type                   103961 non-null   object 
 13

In [3]:
df = df.drop(columns=[ "Unnamed: 0",
                       "user_type" , 
                       "description" ,
                       "title" ,
                       "rent_mode" ,
                       "rent_to_single" ,
                       "rent_type" , 
                       "price_mode", 
                       "credit_mode",
                       "rent_credit_transform",
                       "transformable_price",
                       "transformable_credit",
                       "transformable_credit",
                       "transformable_rent",
                       "transformed_credit",
                       "transformed_rent",
                       "deed_type",
                       "has_business_deed",
                       "total_floors_count",
                       "has_water",
                       "has_electricity",
                       "has_gas",
                       "has_restroom",
                       "building_direction",
                       "floor_material",
                       "property_type",
                       "regular_person_capacity",
                       "extra_person_capacity",
                       "cost_per_extra_person",
                       "rent_price_on_regular_days",
                       "rent_price_on_special_days",
                       "rent_price_at_weekends",
                       "location_radius"
                       ])

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 29 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   cat2_slug                1000000 non-null  object 
 1   cat3_slug                999999 non-null   object 
 2   city_slug                999998 non-null   object 
 3   neighborhood_slug        437139 non-null   object 
 4   created_at_month         1000000 non-null  object 
 5   rent_value               351322 non-null   float64
 6   price_value              568346 non-null   float64
 7   credit_value             352095 non-null   float64
 8   land_size                186396 non-null   float64
 9   building_size            980394 non-null   float64
 10  floor                    541748 non-null   object 
 11  rooms_count              845899 non-null   object 
 12  unit_per_floor           302283 non-null   object 
 13  has_balcony              506411 non-null   

<div dir="rtl">

بجای مختصات از امتیاز محله استفاده میکنیم
(فعلا برای جلو گیری از لیک بعد از ترین تست اسپلیت استفاده میکنیم)</br>

ساخت سطون ماه و سال شمسی و حذف سطون ماه میلادی

In [4]:
# Before splitting or encoding:
df['neighborhood_slug'] = df['neighborhood_slug'].fillna(df['city_slug'])

# Create a unique location key
df['location_key'] = df['city_slug'] + "_" + df['neighborhood_slug']



In [5]:
import jdatetime

df["created_at_month"] = df["created_at_month"].astype("datetime64[ns]")

def gregorian_dt_to_jalali_year_month(dt):
    j = jdatetime.date.fromgregorian(
        year=dt.year,
        month=dt.month,
        day=dt.day
    )
    return pd.Series([j.year, j.month])


df[["created_jalali_year", "created_jalali_month"]] = (
    df["created_at_month"]
    .apply(gregorian_dt_to_jalali_year_month)
)


df["created_jalali_year"] = df["created_jalali_year"].astype(int)
df["created_jalali_month"] = df["created_jalali_month"].astype(int)

In [6]:
df = df.drop(
    columns=[
        "city_slug",
        "neighborhood_slug",
        "created_at_month",
        "location_latitude",
        "location_longitude" 
        ])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 27 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   cat2_slug                1000000 non-null  object 
 1   cat3_slug                999999 non-null   object 
 2   rent_value               351322 non-null   float64
 3   price_value              568346 non-null   float64
 4   credit_value             352095 non-null   float64
 5   land_size                186396 non-null   float64
 6   building_size            980394 non-null   float64
 7   floor                    541748 non-null   object 
 8   rooms_count              845899 non-null   object 
 9   unit_per_floor           302283 non-null   object 
 10  has_balcony              506411 non-null   object 
 11  has_elevator             541749 non-null   object 
 12  has_warehouse            728155 non-null   object 
 13  has_parking              728156 non-null   


## پر کردن داده های خالی

In [7]:
# land_size
# برای ویلایی ها فقط پره پس سایز زمین آپارتمان ها رو با متراژ ساختمان یکی می کنیم
df.loc[df["land_size"].isna(),"land_size"] = df.loc[df["land_size"].isna(),"building_size"]
# حذف آگهی هایی که اندازه زمین ندارند
df = df.dropna(subset=["building_size"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 980394 entries, 0 to 999999
Data columns (total 27 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   cat2_slug                980394 non-null  object 
 1   cat3_slug                980393 non-null  object 
 2   rent_value               351320 non-null  float64
 3   price_value              567064 non-null  float64
 4   credit_value             352093 non-null  float64
 5   land_size                980394 non-null  float64
 6   building_size            980394 non-null  float64
 7   floor                    541748 non-null  object 
 8   rooms_count              845899 non-null  object 
 9   unit_per_floor           302283 non-null  object 
 10  has_balcony              506411 non-null  object 
 11  has_elevator             541749 non-null  object 
 12  has_warehouse            728155 non-null  object 
 13  has_parking              728156 non-null  object 
 14  construct

In [8]:
#floor
#Data is in string so we turn it into int

df['floor'] = df['floor'].astype(str)

#    r'(-?\d+)' means: look for an optional minus sign (-?), followed by digits (\d+)
df['floor'] = df['floor'].str.extract(r'(-?\d+)')[0]

#نال ها را به -2 تبدیل کردیم چون اکثرا زمین های خالی و اجاره روزانه بودن و از جنس بقیه طبقات نیستن
df['floor'] = df['floor'].fillna(-2)

df['floor'] = df['floor'].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 980394 entries, 0 to 999999
Data columns (total 27 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   cat2_slug                980394 non-null  object 
 1   cat3_slug                980393 non-null  object 
 2   rent_value               351320 non-null  float64
 3   price_value              567064 non-null  float64
 4   credit_value             352093 non-null  float64
 5   land_size                980394 non-null  float64
 6   building_size            980394 non-null  float64
 7   floor                    980394 non-null  int64  
 8   rooms_count              845899 non-null  object 
 9   unit_per_floor           302283 non-null  object 
 10  has_balcony              506411 non-null  object 
 11  has_elevator             541749 non-null  object 
 12  has_warehouse            728155 non-null  object 
 13  has_parking              728156 non-null  object 
 14  construct

In [9]:
#rooms_count
# They are Persian so we need to turn them to int

room_mapping = {
    'یک': 1,
    'دو': 2,
    'سه': 3,
    'چهار': 4,
    'پنج یا بیشتر': 5,
    'بدون اتاق': 0
}

df['rooms_count'] = df['rooms_count'].replace(room_mapping)

# استفاده از -1 برای زمین و سوییت ها
df['rooms_count'] = pd.to_numeric(df['rooms_count'], errors='coerce').fillna(-1).astype(int)

df.info()

C:\Users\Rumi\AppData\Local\Temp\ipykernel_19960\619871882.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['rooms_count'] = df['rooms_count'].replace(room_mapping)


<class 'pandas.core.frame.DataFrame'>
Index: 980394 entries, 0 to 999999
Data columns (total 27 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   cat2_slug                980394 non-null  object 
 1   cat3_slug                980393 non-null  object 
 2   rent_value               351320 non-null  float64
 3   price_value              567064 non-null  float64
 4   credit_value             352093 non-null  float64
 5   land_size                980394 non-null  float64
 6   building_size            980394 non-null  float64
 7   floor                    980394 non-null  int64  
 8   rooms_count              980394 non-null  int64  
 9   unit_per_floor           302283 non-null  object 
 10  has_balcony              506411 non-null  object 
 11  has_elevator             541749 non-null  object 
 12  has_warehouse            728155 non-null  object 
 13  has_parking              728156 non-null  object 
 14  construct

In [10]:
#unit_per_floor
#Drop for now 
df = df.drop(columns=["unit_per_floor"])

In [11]:
#پر کدن فیتجر های بولی با فالس
boolean_cols = [
    'has_balcony', 'has_elevator', 'has_warehouse', 'has_parking',
    'is_rebuilt', 'has_security_guard', 'has_barbecue', 
    'has_pool', 'has_jacuzzi', 'has_sauna'
]

df[boolean_cols] = df[boolean_cols].fillna(False).astype(bool)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 980394 entries, 0 to 999999
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   cat2_slug                980394 non-null  object 
 1   cat3_slug                980393 non-null  object 
 2   rent_value               351320 non-null  float64
 3   price_value              567064 non-null  float64
 4   credit_value             352093 non-null  float64
 5   land_size                980394 non-null  float64
 6   building_size            980394 non-null  float64
 7   floor                    980394 non-null  int64  
 8   rooms_count              980394 non-null  int64  
 9   has_balcony              980394 non-null  bool   
 10  has_elevator             980394 non-null  bool   
 11  has_warehouse            980394 non-null  bool   
 12  has_parking              980394 non-null  bool   
 13  construction_year        815826 non-null  object 
 14  is_rebuil

C:\Users\Rumi\AppData\Local\Temp\ipykernel_19960\2041201147.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[boolean_cols] = df[boolean_cols].fillna(False).astype(bool)


In [12]:
#construction_year

#digits are persian, we need to turn to int


persian_digits = '۰۱۲۳۴۵۶۷۸۹'
english_digits = '0123456789'
translation_table = str.maketrans(persian_digits, english_digits)

df['construction_year'] = (
    df['construction_year']
    .astype(str)
    .str.translate(translation_table)
    .str.extract(r'(\d+)')[0]
)

df['construction_year'] = pd.to_numeric(df['construction_year'])

# 4. Fill 'plot-old' nulls

min_year = df['construction_year'].min()

mask_plot_old = (df['cat3_slug'] == 'plot-old') & (df['construction_year'].isna())
df.loc[mask_plot_old, 'construction_year'] = min_year - 1

# 5. Drop remaining nulls and convert to int
df = df.dropna(subset=['construction_year']).copy()
df['construction_year'] = df['construction_year'].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 949384 entries, 1 to 999999
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   cat2_slug                949384 non-null  object 
 1   cat3_slug                949384 non-null  object 
 2   rent_value               350928 non-null  float64
 3   price_value              566655 non-null  float64
 4   credit_value             351697 non-null  float64
 5   land_size                949384 non-null  float64
 6   building_size            949384 non-null  float64
 7   floor                    949384 non-null  int64  
 8   rooms_count              949384 non-null  int64  
 9   has_balcony              949384 non-null  bool   
 10  has_elevator             949384 non-null  bool   
 11  has_warehouse            949384 non-null  bool   
 12  has_parking              949384 non-null  bool   
 13  construction_year        949384 non-null  int64  
 14  is_rebuil

## اضافه کردن عمر سازه

In [13]:
df['property_age'] = df['created_jalali_year'] - df['construction_year']

In [14]:
# has_warm_water_provider 
# has_heating_system 
# has_cooling_system
# cat2_slug 
# drop for now

df1 = df.drop(columns=[
    "has_warm_water_provider", 
    "has_heating_system" ,
     "has_cooling_system",
     "cat2_slug" ])

df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 949384 entries, 1 to 999999
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   cat3_slug             949384 non-null  object 
 1   rent_value            350928 non-null  float64
 2   price_value           566655 non-null  float64
 3   credit_value          351697 non-null  float64
 4   land_size             949384 non-null  float64
 5   building_size         949384 non-null  float64
 6   floor                 949384 non-null  int64  
 7   rooms_count           949384 non-null  int64  
 8   has_balcony           949384 non-null  bool   
 9   has_elevator          949384 non-null  bool   
 10  has_warehouse         949384 non-null  bool   
 11  has_parking           949384 non-null  bool   
 12  construction_year     949384 non-null  int64  
 13  is_rebuilt            949384 non-null  bool   
 14  has_security_guard    949384 non-null  bool   
 15  has_b

In [15]:
bool_cols = df1.select_dtypes(include=['bool']).columns

df1[bool_cols] = df1[bool_cols].astype(int)

df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 949384 entries, 1 to 999999
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   cat3_slug             949384 non-null  object 
 1   rent_value            350928 non-null  float64
 2   price_value           566655 non-null  float64
 3   credit_value          351697 non-null  float64
 4   land_size             949384 non-null  float64
 5   building_size         949384 non-null  float64
 6   floor                 949384 non-null  int64  
 7   rooms_count           949384 non-null  int64  
 8   has_balcony           949384 non-null  int64  
 9   has_elevator          949384 non-null  int64  
 10  has_warehouse         949384 non-null  int64  
 11  has_parking           949384 non-null  int64  
 12  construction_year     949384 non-null  int64  
 13  is_rebuilt            949384 non-null  int64  
 14  has_security_guard    949384 non-null  int64  
 15  has_b

## جدا سازی اجاره و خرید

In [16]:
# RENT 
df_rent = df1[((df1['rent_value'].notna() | df1['credit_value'].notna()) & df1['price_value'].isna())].copy()

# SELL
df_sell = df1[(df1['price_value'].notna() & (df1['rent_value'].isna()))].copy()

df_sell = df_sell.drop(columns=['rent_value', 'credit_value'])
df_rent = df_rent.drop(columns=['price_value'])

df_rent[['rent_value', 'credit_value']] = df_rent[['rent_value', 'credit_value']].fillna(0)

#delete index
df_sell = df_sell.drop(columns=['Unnamed: 0'], errors='ignore')
df_rent = df_rent.drop(columns=['Unnamed: 0'], errors='ignore')


## حذف داده های پرت

In [17]:
# --- 1. CLEANING df_sell ---
# Filter price_value: 1 milion <= price <= 10 trilion
price_min = 1e6
price_max = 10e12

df_sell = df_sell[
    (df_sell['price_value'] >= price_min) & 
    (df_sell['price_value'] <= price_max)
].copy()


# Filter rent_value <= 10 billion
# Filter credit_value <= 1 trilion
rent_max = 10e9
credit_max = 1e12

df_rent = df_rent[
    (df_rent['rent_value'] <= rent_max) & 
    (df_rent['credit_value'] <= credit_max)
].copy()

# Print results to see how many rows remain
print(f"Remaining Sell listings: {df_sell.shape[0]}")
print(f"Remaining Rent listings: {df_rent.shape[0]}")


Remaining Sell listings: 557336
Remaining Rent listings: 351201


## تبدیل رهن و اجاره به رهن کامل

In [18]:
df_rent['target_credit'] = df_rent['credit_value'] + df_rent['rent_value'] * 30

df_rent = df_rent.drop(columns=["rent_value", "credit_value"])



## Encoding cat3

In [19]:
df_rent.cat3_slug.value_counts()

cat3_slug
apartment-rent                        211395
house-villa-rent                       64410
shop-rent                              45623
office-rent                            21172
industry-agriculture-business-rent      8595
plot-old                                   3
shop-sell                                  2
office-sell                                1
Name: count, dtype: int64

In [20]:
# Define allowed categories for Rent
allowed_rent = ['apartment-rent', 'house-villa-rent', 'shop-rent', 'office-rent', 'industry-agriculture-business-rent']
df_rent = df_rent[df_rent['cat3_slug'].isin(allowed_rent)].copy()

# Define allowed categories for Sell
allowed_sell = ['apartment-sell', 'house-villa-sell', 'plot-old', 'shop-sell', 'industry-agriculture-business-sell', 'office-sell']
df_sell = df_sell[df_sell['cat3_slug'].isin(allowed_sell)].copy()

In [21]:
df_rent.cat3_slug.value_counts()

cat3_slug
apartment-rent                        211395
house-villa-rent                       64410
shop-rent                              45623
office-rent                            21172
industry-agriculture-business-rent      8595
Name: count, dtype: int64

In [22]:
from sklearn.preprocessing import LabelEncoder

# Initialize encoders (using separate ones for safety, though one is fine)
le_rent = LabelEncoder()
le_sell = LabelEncoder()

# Apply to the columns
df_rent['cat3_slug'] = le_rent.fit_transform(df_rent['cat3_slug'])
df_sell['cat3_slug'] = le_sell.fit_transform(df_sell['cat3_slug'])

# Quick check: print the mapping so you know which number is which
print("Rent Mapping:", dict(zip(le_rent.classes_, le_rent.transform(le_rent.classes_))))
print("Sell Mapping:", dict(zip(le_sell.classes_, le_sell.transform(le_sell.classes_))))

Rent Mapping: {'apartment-rent': np.int64(0), 'house-villa-rent': np.int64(1), 'industry-agriculture-business-rent': np.int64(2), 'office-rent': np.int64(3), 'shop-rent': np.int64(4)}
Sell Mapping: {'apartment-sell': np.int64(0), 'house-villa-sell': np.int64(1), 'industry-agriculture-business-sell': np.int64(2), 'office-sell': np.int64(3), 'plot-old': np.int64(4), 'shop-sell': np.int64(5)}


## حذف بیشتر داده پرت

In [23]:
# 1. Calculate quantiles for Sell
q_low_sell = df_sell["price_value"].quantile(0.1)
q_hi_sell  = df_sell["price_value"].quantile(0.9996)

# 2. Filter the Sell dataframe
df_sell_filtered = df_sell[(df_sell["price_value"] < q_hi_sell) & (df_sell["price_value"] > q_low_sell)]

# 3. Print results to check the "Common Sense" boundaries
print(f"Sell original: {len(df_sell)} rows")
print(f"Sell filtered: {len(df_sell_filtered)} rows")
print(f"Cutoff values: {q_low_sell:,.0f} to {q_hi_sell:,.0f} ريال")

# Calculate quantiles for Rent (Total converted credit)
q_low_rent = df_rent["target_credit"].quantile(0.1)
q_hi_rent  = df_rent["target_credit"].quantile(0.9996)

# Filter the Rent dataframe
df_rent_filtered = df_rent[(df_rent["target_credit"] < q_hi_rent) & (df_rent["target_credit"] > q_low_rent)]

print(f"Rent original: {len(df_rent)} rows")
print(f"Rent filtered: {len(df_rent_filtered)} rows")
print(f"Cutoff values: {q_low_rent:,.0f} to {q_hi_rent:,.0f} ريال")

Sell original: 557336 rows
Sell filtered: 499154 rows
Cutoff values: 750,000,000 to 2,153,300,000,000 ريال
Rent original: 351195 rows
Rent filtered: 315307 rows
Cutoff values: 155,000,000 to 131,522,400,000 ريال


## Train test Split

In [24]:
# from sklearn.model_selection import train_test_split

# # --- 1. Split SELL Data ---
# X_sell = df_sell.drop(columns=['price_value'])
# y_sell = df_sell['price_value']

# X_sell_train, X_sell_test, y_sell_train, y_sell_test = train_test_split(
#     X_sell, y_sell, test_size=0.2, random_state=42
# )

# # --- 2. Split RENT Data ---
# X_rent = df_rent.drop(columns=['target_credit'])
# y_rent = df_rent['target_credit']

# X_rent_train, X_rent_test, y_rent_train, y_rent_test = train_test_split(
#     X_rent, y_rent, test_size=0.2, random_state=42
# )

# print(f"Sell Train Shape: {X_sell_train.shape}")
# print(f"Rent Train Shape: {X_rent_train.shape}")

In [25]:
from sklearn.model_selection import train_test_split

# --- 1. Split SELL Data (using filtered df) ---
# Ensure you are using the dataframe created by the 1%-99% quantile filter
X_sell = df_sell_filtered.drop(columns=['price_value'])
y_sell = df_sell_filtered['price_value']

X_sell_train, X_sell_test, y_sell_train, y_sell_test = train_test_split(
    X_sell, y_sell, test_size=0.2, random_state=42
)

# --- 2. Split RENT Data (using filtered df) ---
# Ensure you are using the dataframe created by the 1%-99% quantile filter
X_rent = df_rent_filtered.drop(columns=['target_credit'])
y_rent = df_rent_filtered['target_credit']

X_rent_train, X_rent_test, y_rent_train, y_rent_test = train_test_split(
    X_rent, y_rent, test_size=0.2, random_state=42
)

print(f"Sell Filtered Train Shape: {X_sell_train.shape}")
print(f"Rent Filtered Train Shape: {X_rent_train.shape}")

Sell Filtered Train Shape: (399323, 20)
Rent Filtered Train Shape: (252245, 20)


## log(+1) scale Prices


In [26]:

# Log transform for Sell
y_sell_train_log = np.log1p(y_sell_train)
y_sell_test_log = np.log1p(y_sell_test)

# Log transform for Rent
y_rent_train_log = np.log1p(y_rent_train)
y_rent_test_log = np.log1p(y_rent_test)

## اضافه کردن location_score

In [27]:
sell_loc_means = y_sell_train_log.groupby(X_sell_train['location_key']).mean()

# 3. Add that average as a new numeric column 'location_score'
X_sell_train['location_score'] = X_sell_train['location_key'].map(sell_loc_means)
X_sell_test['location_score'] = X_sell_test['location_key'].map(sell_loc_means)

# 4. If a neighborhood exists in Test but not in Train, use the overall average price
X_sell_test['location_score'] = X_sell_test['location_score'].fillna(y_sell_train_log.mean())

rent_loc_means = y_rent_train_log.groupby(X_rent_train['location_key']).mean()

# 3. Add that average as a new numeric column 'location_score'
X_rent_train['location_score'] = X_rent_train['location_key'].map(rent_loc_means)
X_rent_test['location_score'] = X_rent_test['location_key'].map(rent_loc_means)

# 4. Fill missing scores with the overall average rent
X_rent_test['location_score'] = X_rent_test['location_score'].fillna(y_rent_train_log.mean())

## اضافه کردن کیفیت محل بر اساس قیمت بر متر

In [28]:
# 1. Calculate raw PPM on TRAIN only
X_sell_train['temp_ppm'] = y_sell_train / X_sell_train['building_size']

# 2. Group by location and calculate the mean of the LOG of PPM
# We apply log1p ONLY to the 'temp_ppm' column
ppm_map = np.log1p(X_sell_train['temp_ppm']).groupby(X_sell_train['location_key']).mean()

# 3. Map this "Neighborhood Quality" to both sets
X_sell_train['loc_quality_index'] = X_sell_train['location_key'].map(ppm_map)
X_sell_test['loc_quality_index'] = X_sell_test['location_key'].map(ppm_map)

# 4. Fill missing values (neighborhoods in test not in train) with the global average
global_avg_log_ppm = np.log1p(X_sell_train['temp_ppm']).mean()
X_sell_test['loc_quality_index'] = X_sell_test['loc_quality_index'].fillna(global_avg_log_ppm)


In [29]:
# Drop the text columns from both sets
X_sell_train = X_sell_train.drop(columns=['location_key', 'city_slug', 'neighborhood_slug','temp_ppm'], errors='ignore')
X_sell_test = X_sell_test.drop(columns=['location_key', 'city_slug', 'neighborhood_slug', 'temp_ppm'], errors='ignore')

X_rent_train = X_rent_train.drop(columns=['location_key', 'city_slug', 'neighborhood_slug'], errors='ignore')
X_rent_test = X_rent_test.drop(columns=['location_key', 'city_slug', 'neighborhood_slug'], errors='ignore')

In [30]:
X_sell_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 399323 entries, 543671 to 244227
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   cat3_slug             399323 non-null  int64  
 1   land_size             399323 non-null  float64
 2   building_size         399323 non-null  float64
 3   floor                 399323 non-null  int64  
 4   rooms_count           399323 non-null  int64  
 5   has_balcony           399323 non-null  int64  
 6   has_elevator          399323 non-null  int64  
 7   has_warehouse         399323 non-null  int64  
 8   has_parking           399323 non-null  int64  
 9   construction_year     399323 non-null  int64  
 10  is_rebuilt            399323 non-null  int64  
 11  has_security_guard    399323 non-null  int64  
 12  has_barbecue          399323 non-null  int64  
 13  has_pool              399323 non-null  int64  
 14  has_jacuzzi           399323 non-null  int64  
 15  

In [38]:
import xgboost as xgb
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Initialize and train Sell Model
xgb_sell = xgb.XGBRegressor(
    n_estimators=2000,      
    learning_rate=0.03,     
    max_depth=10,           
    subsample=0.9,
    colsample_bytree=0.9,
    tree_method='hist',     
    random_state=42
)

xgb_sell.fit(
    X_sell_train, y_sell_train_log,
    eval_set=[(X_sell_test, y_sell_test_log)],
    verbose=False 
)

sell_preds_log = xgb_sell.predict(X_sell_test)

# Calculate Sell Metrics (Log Space)
sell_r2 = r2_score(y_sell_test_log, sell_preds_log)
sell_mae_log = mean_absolute_error(y_sell_test_log, sell_preds_log)
sell_mse_log = mean_squared_error(y_sell_test_log, sell_preds_log)

# Calculate Sell Metrics (Real ريال)
sell_preds_real = np.expm1(sell_preds_log)
sell_actual_real = np.expm1(y_sell_test_log)
sell_mae_real = mean_absolute_error(sell_actual_real, sell_preds_real)

print(f"--- SELL RESULTS ---")
print(f"R2 Score (Log): {sell_r2:.4f}")
print(f"MAE (Log):      {sell_mae_log:.4f}")
print(f"MSE (Log):      {sell_mse_log:.4f}")
print(f"Average Error:  {sell_mae_real:,.0f} ريال")
print("-" * 20)

# Initialize and train Rent Model
xgb_rent = xgb.XGBRegressor(
    n_estimators=2000,      
    learning_rate=0.03,     
    max_depth=10,           
    subsample=0.9,
    colsample_bytree=0.9,
    tree_method='hist',     
    random_state=42
)

xgb_rent.fit(
    X_rent_train, y_rent_train_log,
    eval_set=[(X_rent_test, y_rent_test_log)],
    verbose=False
)

rent_preds_log = xgb_rent.predict(X_rent_test)
# Calculate Rent Metrics (Log Space)
rent_r2 = r2_score(y_rent_test_log, rent_preds_log)
rent_mae_log = mean_absolute_error(y_rent_test_log, rent_preds_log)
rent_mse_log = mean_squared_error(y_rent_test_log, rent_preds_log)

# Calculate Rent Metrics (Real ريال)
rent_preds_real = np.expm1(rent_preds_log)
rent_actual_real = np.expm1(y_rent_test_log)
rent_mae_real = mean_absolute_error(rent_actual_real, rent_preds_real)

print(f"--- RENT RESULTS ---")
print(f"R2 Score (Log): {rent_r2:.4f}")
print(f"MAE (Log):      {rent_mae_log:.4f}")
print(f"MSE (Log):      {rent_mse_log:.4f}")
print(f"Average Error:  {rent_mae_real:,.0f} ريال")

--- SELL RESULTS ---
R2 Score (Log): 0.6799
MAE (Log):      0.3298
MSE (Log):      0.3131
Average Error:  3,858,627,491 ريال
--------------------
--- RENT RESULTS ---
R2 Score (Log): 0.7460
MAE (Log):      0.2783
MSE (Log):      0.2083
Average Error:  411,434,568 ريال


In [39]:
import numpy as np
from sklearn.metrics import r2_score

# --- Calculate R2 for SELL Train Set ---
sell_train_preds_log = xgb_sell.predict(X_sell_train)
sell_train_r2 = r2_score(y_sell_train_log, sell_train_preds_log)
print(f"Sell R2 Score (Train): {sell_train_r2:.4f}")

# --- Calculate R2 for RENT Train Set ---
rent_train_preds_log = xgb_rent.predict(X_rent_train)
rent_train_r2 = r2_score(y_rent_train_log, rent_train_preds_log)
print(f"Rent R2 Score (Train): {rent_train_r2:.4f}")

Sell R2 Score (Train): 0.8453
Rent R2 Score (Train): 0.9117


In [33]:
import xgboost as xgb
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# --- 1. SELL MODEL ---
xgb_sell = xgb.XGBRegressor(
    n_estimators=2000, # Increased estimators, early stopping will decide actual number
    learning_rate=0.03, # Slightly reduced learning rate
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    tree_method='hist'
)

xgb_sell.fit(
    X_sell_train, y_sell_train_log,
    eval_set=[(X_sell_test, y_sell_test_log)],
    verbose=False
)

sell_preds_log = xgb_sell.predict(X_sell_test)

# Calculate Sell Metrics (Log Space)
sell_r2 = r2_score(y_sell_test_log, sell_preds_log)
sell_mae_log = mean_absolute_error(y_sell_test_log, sell_preds_log)
sell_mse_log = mean_squared_error(y_sell_test_log, sell_preds_log)

# Calculate Sell Metrics (Real ريال)
sell_preds_real = np.expm1(sell_preds_log)
sell_actual_real = np.expm1(y_sell_test_log)
sell_mae_real = mean_absolute_error(sell_actual_real, sell_preds_real)

print(f"--- SELL RESULTS ---")
print(f"R2 Score (Log): {sell_r2:.4f}")
print(f"MAE (Log):      {sell_mae_log:.4f}")
print(f"MSE (Log):      {sell_mse_log:.4f}")
print(f"Average Error:  {sell_mae_real:,.0f} ريال")
print("-" * 20)


# --- 2. RENT MODEL ---
xgb_rent = xgb.XGBRegressor(
    n_estimators=2000, # Increased estimators, early stopping will decide actual number
    learning_rate=0.03, # Slightly reduced learning rate
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    tree_method='hist'
)

xgb_rent.fit(
    X_rent_train, y_rent_train_log,
    eval_set=[(X_rent_test, y_rent_test_log)],
    verbose=False
)

rent_preds_log = xgb_rent.predict(X_rent_test)

# Calculate Rent Metrics (Log Space)
rent_r2 = r2_score(y_rent_test_log, rent_preds_log)
rent_mae_log = mean_absolute_error(y_rent_test_log, rent_preds_log)
rent_mse_log = mean_squared_error(y_rent_test_log, rent_preds_log)

# Calculate Rent Metrics (Real ريال)
rent_preds_real = np.expm1(rent_preds_log)
rent_actual_real = np.expm1(y_rent_test_log)
rent_mae_real = mean_absolute_error(rent_actual_real, rent_preds_real)

print(f"--- RENT RESULTS ---")
print(f"R2 Score (Log): {rent_r2:.4f}")
print(f"MAE (Log):      {rent_mae_log:.4f}")
print(f"MSE (Log):      {rent_mse_log:.4f}")
print(f"Average Error:  {rent_mae_real:,.0f} ريال")

--- SELL RESULTS ---
R2 Score (Log): 0.6831
MAE (Log):      0.3344
MSE (Log):      0.3100
Average Error:  3,861,458,594 ريال
--------------------
--- RENT RESULTS ---
R2 Score (Log): 0.7535
MAE (Log):      0.2764
MSE (Log):      0.2022
Average Error:  408,383,050 ريال


In [34]:
import numpy as np
from sklearn.metrics import r2_score

# --- Calculate R2 for SELL Train Set ---
sell_train_preds_log = xgb_sell.predict(X_sell_train)
sell_train_r2 = r2_score(y_sell_train_log, sell_train_preds_log)
print(f"Sell R2 Score (Train): {sell_train_r2:.4f}")

# --- Calculate R2 for RENT Train Set ---
rent_train_preds_log = xgb_rent.predict(X_rent_train)
rent_train_r2 = r2_score(y_rent_train_log, rent_train_preds_log)
print(f"Rent R2 Score (Train): {rent_train_r2:.4f}")

Sell R2 Score (Train): 0.7705
Rent R2 Score (Train): 0.8458
